## Project to automate api access 
+ The goal of this project is to start automating api calls based on Democratic Party Presidential candidates
+ I am accessing the Newsapi, it stores metadata form over 130 publications
    + Among the data I will be scraping to analyze are the sources themselves, author, first 250 words of content, and summary of the article
+ The goal here will be to  run NLP on different poltical topics
    + I 

In [50]:
import pandas as pd
import config
import math
from newsapi import NewsApiClient


# Init
newsapi = NewsApiClient(api_key=config.api_key)


# Grab all sources
+ read through available sources list and make df storing domain and source name

In [9]:
sources = newsapi.get_sources()
new_orgs = sources["sources"]
my_sources = {}
for i, x in enumerate(new_orgs):
    my_sources[i] = (x['id'])
domains = sources["sources"]
my_domains = {}
for i, x in enumerate(domains):
    my_domains[i] = (x['url'])
sources = pd.Series(my_sources).to_frame("sources")
domains = pd.Series(my_domains).to_frame("domains")
query_keys_df = domains.join(sources)
print(query_keys_df.to_string())

                                        domains                      sources
0                        https://abcnews.go.com                     abc-news
1                    http://www.abc.net.au/news                  abc-news-au
2                    https://www.aftenposten.no                  aftenposten
3                      http://www.aljazeera.com           al-jazeera-english
4                            http://www.ansa.it                         ansa
5                         http://www.argaam.com                       argaam
6                        http://arstechnica.com                 ars-technica
7                        https://arynews.tv/ud/                     ary-news
8                           https://apnews.com/             associated-press
9                            http://www.afr.com  australian-financial-review
10                        https://www.axios.com                        axios
11                    http://www.bbc.co.uk/news                     bbc-news

In [40]:
18,20,22,23,24,39,41,44,62,82,83,93,98,99,111,114,117,119,121,124,127,128,132

'abc-news,al-jazeera-english,associated-press,bbc-news,bloomberg,breitbart-news,business-insider,buzzfeed,cbs-news,cnbc'

## Choosing data sources
+ Lets attempt to grab some sources from different geographic locations as well as different idological perspectives

+ categorizing news sources
    + Traditional TV MSM
        +  http://us.cnn.com   
        +  http://www.cnbc.com 
        +  http://www.foxnews.com  
        +  http://www.msnbc.com  
        +  https://abcnews.go.com  
        +  http://www.nbcnews.com  
    + Traditional publications 
        +  http://www.nytimes.com  
        +  https://www.washingtonpost.com 
        
    + Internet Sources
        +  http://www.huffingtonpost.com 
        +  https://www.politico.com
        +  http://www.breitbart.com 
        +  https://news.google.com 
        +  https://www.buzzfeed.com 
        +  https://news.vice.com  
    + Financial publications
        +  http://www.economist.com
        +  http://www.bloomberg.com 
        +  http://www.businessinsider.com 
        +  http://www.wsj.com
        +  http://fortune.com  
        
    + News aggregators
        +  https://apnews.com/ 
        +  http://www.reuters.com 
    + foreign reporting
         + http://www.aljazeera.com  
         + http://www.bbc.co.uk/news   
         + https://www.jpost.com/  
         + http://timesofindia.indiatimes.com 
         + https://russian.rt.com 
         + https://www.theguardian.com/uk 
         + http://www.independent.co.uk  
         + http://www.telegraph.co.uk  


 



In [44]:
a= query_keys_df.iloc[[0,3,8,11,16,17,18,20,22,23,24,39,41,44,62,82,83,93,98,99,111,114,117,119,121,124,127,128,132],[1]]
list_sources =a["sources"].tolist()
myString = ",".join(list_sources)
myString

'abc-news,al-jazeera-english,associated-press,bbc-news,bloomberg,breitbart-news,business-insider,buzzfeed,cbs-news,cnbc,cnn,fortune,fox-news,google-news,independent,national-review,nbc-news,politico,reuters,rt,the-economist,the-guardian-uk,the-huffington-post,the-jerusalem-post,the-new-york-times,the-telegraph,the-wall-street-journal,the-washington-post,vice-news'

In [ ]:
+ You can see the string which we will insert into sources above

## Now lets begin process of automating query calls
+ After the first call we can see our total results for the day, which will allow us to make subsequent calls.
+ first lets build function to clean query returns

In [62]:
def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

In [46]:
bernie_sanders_list=[]
# Make first call
all_articles = newsapi.get_everything(q='Bernie Sanders',
                                      sources=myString,
                                      domains='https://apnews.com/,http://www.nytimes.com',
                                      language='en',
                                      from_param='2019-18-01',
                                      sort_by='relevancy',
                                      page_size=100,
                                      page=1)
total_pages = math.ceil(all_articles["totalResults"]/100)

all_articles = clean_query(all_articles)
bernie_sanders_list.append(all_articles)

for page in range(2,total_pages+1):
    all_articles = newsapi.get_everything(q='Bernie Sanders',
                                      sources=myString,
                                      domains='https://apnews.com/,http://www.nytimes.com',
                                      language='en',
                                      from_param='2019-18-01',
                                      sort_by='relevancy',
                                      page_size=100,
                                      page=2)
    ran_query = clean_query(all_articles)
    #ran_query_df = pd.DataFrame(ran_query['articles'])
    bernie_sanders_list.append(ran_query)


In [84]:
df = pd.concat(bernie_sanders_list)

TypeError: concat() got an unexpected keyword argument 'reindex'

In [88]:
df.reset_index(drop=True)

,author,content,description,publishedAt,source,title,url
0,"Jeff Stein, The Washington Post","WASHINGTON - Sen. Bernie Sanders, I-Vt., will ...","WASHINGTON - Sen. Bernie Sanders, I-Vt., will ...",2019-01-31,The Washington Post,Sanders to propose dramatic expansion in estat...,http://www.washingtonpost.com/us-policy/2019/0...
1,Paul Steinhauser,Former Vice President Joe Biden and Vermont Se...,"Biden, Sanders, Harris, Warren top 2020 Dem fi...",2019-02-12,Fox News,"Biden, Sanders, Harris, Warren top 2020 Dem fi...",https://www.foxnews.com/politics/biden-sanders...
2,Justin Haskins,Venezuela is once again in a state of economic...,Democrats now pushing many of the same sociali...,2019-01-27,Fox News,Democrats now pushing many of the same sociali...,https://www.foxnews.com/opinion/democrats-now-...
3,David Smith in Washington,Diners and town halls. Iowa and New Hampshire....,More and more Democrats vying for the White Ho...,2019-01-27,The Guardian (AU),Late-night with Democrats: how going viral may...,https://www.theguardian.com/us-news/2019/jan/2...
4,Daniel Marans,"SIOUX CITY, Iowa When Sen. Kirsten Gillibrand ...",The presidential prospect is touting her polit...,2019-01-20,The Huffington Post,Kirsten Gillibrand Highlights Rural Roots In I...,https://www.huffingtonpost.com/entry/kirsten-g...
5,The Associated Press,Sen. Kamala Harris returned to South Carolina ...,Sen. Kamala Harris returned to South Carolina ...,2019-01-26,ABC News,"Tapping into sorority network, Kamala Harris r...",https://abcnews.go.com/Politics/wireStory/tapp...
6,nmccaskill@politico.com (Nolan D. McCaskill),"Dominick Reuter/AFP/Getty Images\r\nNEWARK, N....",The intentional lack of stage management was s...,2019-02-01,Politico,Booker holds court in Newark — and breaks with...,https://www.politico.com/story/2019/02/01/cory...
7,sbland@politico.com (Scott Bland),Online fundraising is poised to play a critica...,A new FEC filing gives an early look at Democr...,2019-02-01,Politico,Warren raised $299K online on launch day of ex...,https://www.politico.com/story/2019/02/01/eliz...
8,Tim Walker,"Good morning, Im Tim Walker with todays essent...",Democratic 2020 hopefuls head south to mark ML...,2019-01-21,The Guardian (AU),Monday US briefing: Trump shutdown concessions...,https://www.theguardian.com/us-news/2019/jan/2...
9,Cristina Tzintzun Ramirez,"When all is said and done, there may be as man...",Just a few years ago Sanders' call for 'Medica...,2019-02-03,Al Jazeera English,Bernie's healthcare policies already won the 2...,https://www.aljazeera.com/indepth/opinion/bern...


In [ ]:
#sqlalchemy tutorial q_sql read_sql    df.tosql 